In [8]:
import os
from pprint import pprint
from pdfrw import PdfReader
from pdfminer.layout import LTTextBoxHorizontal, LAParams
from pdfminer.pdfparser import PDFParser, PDFDocument
from pdfminer.pdfinterp import PDFTextExtractionNotAllowed
from pdfminer.converter import PDFPageAggregator, TextConverter
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from io import StringIO
import unicodedata

In [9]:
def all_pdfs(src_path='examples'):
    print('=' * 80)
    print(src_path)
    pdfs = [s for s in os.listdir(src_path) if s .endswith('.pdf')]
    print(pdfs)
    return [os.path.join(src_path, e) for e in pdfs]

def empty_info():
    info = dict(
        title = '',
        keywords = [],
        abstract = '',
    )
    return info

def pdf_parser(fp):
    # Prepare doc
    praser = PDFParser(fp)
    doc = PDFDocument()
    praser.set_document(doc)
    doc.set_parser(praser)
    doc.initialize()
    
    # Rude assertion
    assert(doc.is_extractable)
    
    # Prepare components
    rsrcmgr = PDFResourceManager()
    laparams = LAParams()
    
    # Bound device and interpreter
    device = PDFPageAggregator(rsrcmgr, laparams=laparams)
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    
    return doc, interpreter, device

def paper_cls(pdf_path):
    info = PdfReader(pdf_path).Info
    if info['/Creator'] == '(untitled)':
        return 0

In [10]:
pdf_path = 'examples\ieee_2.pdf'

pdf_path = 'C:\\Users\\liste\\OneDrive\\Documents\\schorlar\\! VisualFeatureExtractionfromVoxel-Weighted-Averaging-fMRI.pdf'

def parse_paper_ieee(pdf_path):

    info = empty_info()

    with open(pdf_path, 'rb') as fp:
        
        doc, interpreter, device = pdf_parser(fp)
        
        # Read infos
        # Only search on first page
        page_limit = 1
        is_title = False
        for j, page in enumerate(doc.get_pages()):
            # Break when page_limit is reached
            if not(j < page_limit):
                break
            # Get layout
            interpreter.process_page(page)
            layout = device.get_result()
            # For each box
            for box in layout:
                # If not LTTextBoxHorizontal, it should be passed
                if not isinstance(box, LTTextBoxHorizontal):
                    continue
                # Get content
                content = box.get_text()
                # print('-' * 80)
                # print(content)
                
                if is_title:
                    info['title'] = content.replace('\n', ' ').strip()
                    is_title = False
                if content.startswith('IEEE'):
                    is_title = True
                    
                # Remove short dash '—'
                content = content.replace('—', '')
                
                # Record abstract
                if content.startswith('Abstract'):
                    content = content.replace('Abstract', '')
                    info['abstract'] = content.replace('\n', ' ').replace('- ', '').strip()
                    
                # Record keywords
                # Deal with several 'Index Terms' feature strings
                content = content.replace('Index Terms', 'IndexTerms')
                if content.startswith('IndexTerms'):
                    keywords = content.replace('IndexTerms', '').replace('\n', '').split(',')
                    if keywords[-1].endswith('.'):
                        keywords[-1] = keywords[-1][:-1]
                    info['keywords'] = [e.strip().lower() for e in keywords]
    return info

parse_paper_ieee(pdf_path)

{'title': 'Visual Feature Extraction From Voxel-Weighted Averaging of Stimulus Images in 2 fMRI Studies',
 'keywords': ['bayesian estimation',
  'component analysis',
  'fmri',
  'generalized linear models',
  'imaging',
  'voxel'],
 'abstract': 'Multiple studies have provided evidence for distributed object representation in the brain, with several recent experiments leveraging basis function estimates for partial image reconstruction from fMRI data. Using a novel combination of statistical decomposition, generalized linear models, and stimulus averaging on previously examined image sets and Bayesian regression of recorded fMRI activity during presentation of these data sets, we identify a subset of relevant voxels that appear to code for covarying object features. Using a technique we term “voxel-weighted averaging,” we isolate image ﬁlters that these voxels appear to implement. The results, though very cursory, appear to have signiﬁcant implications for hierarchical and deep-learnin

In [11]:
pdf_path = 'examples\elsevier_2.pdf'
pdf_path = 'C:\\Users\\liste\\OneDrive\\Documents\\schorlar\\!Improved estimation of subject-level functional connectivity using full and partial correlation with empirical Bayes shrinkage.pdf'
pdf_path = 'C:\\Users\\liste\\OneDrive\\Documents\\schorlar\\1-s2.0-S1053811916301914-main.pdf'
pdf_path = 'C:\\Users\\liste\\OneDrive\\Documents\\schorlar\\10.1093@cercor@bhy123.pdf'
pdf_path = 'C:\\Users\\liste\\OneDrive\\Documents\\schorlar\\1-s2.0-S0896627306005861-main.pdf'
pdf_path = 'C:\\Users\\liste\\OneDrive\\Documents\\schorlar\\1-s2.0-S0028393216300173-main.pdf'

def parse_paper_elsevier(pdf_path):
    info = empty_info()
    info['title'] = PdfReader(pdf_path).Info['/Title'][1:-1]

    with open(pdf_path, 'rb') as fp:
        
        doc, interpreter, device = pdf_parser(fp)

        # Read infos
        # Only search on first page
        page_limit = 2
        is_abstract = False
        for j, page in enumerate(doc.get_pages()):
            # Break when page_limit is reached
            if not(j < page_limit):
                break
            # Get layout
            interpreter.process_page(page)
            layout = device.get_result()
            # For each box
            for box in layout:
                # If not LTTextBoxHorizontal, it should be passed
                if not isinstance(box, LTTextBoxHorizontal):
                    continue
                # Get content
                content = box.get_text()
                # print('-' * 80)
                # print(content)
                # Record abstract
                # If ready to read abstract, read abstract if not match known distractor
                if all([is_abstract,
                        not content.startswith('Article history'),
                        not content.startswith('Keywords'),
                        not content.startswith('Introduction'),
                        not content.startswith('1.')
                       ]):
                    info['abstract'] = ''.join(content).replace('\n', ' ').strip()
                    is_abstract = False
                # If match abstract feature string, ready to read abstract
                if any([content.lower().startswith('a b s t r a c t'),
                        content.lower().startswith('abstract'),
                        content.lower().startswith('summary')]):
                    is_abstract = True
                # Record keywords
                content = content.replace('Key words', 'Keywords').replace(',', '\n')
                if content.startswith('Keywords'):
                    info['keywords'] = [e.strip().lower() for e in content.split('\n') if e][1:]

    return info

parse_paper_elsevier(pdf_path)

{'title': 'A meta-analysis of fMRI decoding_ Quantifying influences on human visual population codes',
 'keywords': ['decoding',
  'mvpa',
  'patterns',
  'meta-analysis',
  'vision',
  'objects'],
 'abstract': 'Information in the human visual system is encoded in the activity of distributed populations of neurons, which in turn is reﬂected in functional magnetic resonance imaging (fMRI) data. Over the last ﬁfteen years, activity patterns underlying a variety of perceptual features and objects have been decoded from the brains of participants in fMRI scans. Through a novel multi-study meta-analysis, we have analyzed and modeled relations between decoding strength in the visual ventral stream, and stimulus and methodological variables that differ across studies. We report ﬁndings that suggest: (i) several organi- zational principles of the ventral stream, including a gradient of pattern granulation and an increasing abstraction of neural representations as one proceeds anteriorly; (ii) 

In [12]:
def pdf_info(pdf_path):
    info = PdfReader(pdf_path).Info
    print(info)
    if info['/Creator'] == '(Elsevier)':
        return _elsevier(pdf_path)
    return _ieee(pdf_path)
    return None

def _elsevier(pdf_path):
    print('[-- It is an elsevier paper. --]')
    info = parse_paper_elsevier(pdf_path)
    return info

def _ieee(pdf_path):
    print('[-- It is an ieee paper. --]')
    info = parse_paper_ieee(pdf_path)
    return info

In [13]:
for pdf_path in all_pdfs('C:\\Users\\liste\\OneDrive\\Documents\\schorlar'):
    print('-'* 80)
    print(pdf_path)
    info = pdf_info(pdf_path)
    if info:
        pprint(info)

C:\Users\liste\OneDrive\Documents\schorlar
['! VisualFeatureExtractionfromVoxel-Weighted-Averaging-fMRI.pdf', '!10.1093@cercor@bhy123.pdf', '!Improved estimation of subject-level functional connectivity using full and partial correlation with empirical Bayes shrinkage.pdf', '!Integrating theoretical models with functional neuroimaging.pdf', '0102181v1.pdf', '07073970.pdf', '07264993.pdf', '07468569.pdf', '08025614.pdf', '08051252.pdf', '08269806.pdf', '08370896.pdf', '1-s2.0-S0010945217303258-main.pdf', '1-s2.0-S0028393216300173-main.pdf', '1-s2.0-S0028393217300593-main.pdf', '1-s2.0-S0042698909004751-main.pdf', '1-s2.0-S0893608016301800-main.pdf', '1-s2.0-S0896627306005861-main.pdf', '1-s2.0-S089662731830477X-main.pdf', '1-s2.0-S0925231215017634-main.pdf', '1-s2.0-S1053810015000355-main.pdf', '1-s2.0-S1053811907001073-main.pdf', '1-s2.0-S1053811911013012-main.pdf', '1-s2.0-S1053811912004429-main.pdf', '1-s2.0-S1053811913002851-main.pdf', '1-s2.0-S1053811914001220-main.pdf', '1-s2.0-S1

{'abstract': '', 'keywords': [], 'title': ''}
--------------------------------------------------------------------------------
C:\Users\liste\OneDrive\Documents\schorlar\!Improved estimation of subject-level functional connectivity using full and partial correlation with empirical Bayes shrinkage.pdf
{'/Author': '(Amanda F. Mejia)', '/CreationDate': "(D:20180414203338+05'30')", '/Creator': '(Elsevier)', '/CrossMarkDomains[1]': '(sciencedirect.com)', '/CrossMarkDomains[2]': '(elsevier.com)', '/CrossmarkDomainExclusive': '(true)', '/CrossmarkMajorVersionDate': '(2010-04-23)', '/ElsevierWebPDFSpecifications': '(6.5)', '/ModDate': "(D:20190116174106+08'00')", '/Producer': '(Acrobat Distiller 8.1.0 \\(Windows\\))', '/Subject': '(NeuroImage, 172 \\(2018\\) 478-491. doi:10.1016/j.neuroimage.2018.01.029)', '/Title': '(Improved estimation of subject-level functional connectivity using full and partial correlation with empirical Bayes shrinkage)', '/doi': '(10.1016/j.neuroimage.2018.01.029)', '/

{'abstract': 'Linear Gaussian state-space models are ubiquitous in signal '
             'processing, and an important procedure is that of estimating '
             'system parameters from observed data. Rather than making a '
             'single point estimate, it is often desirable to conduct Bayesian '
             'learning, in which the entire posterior distribution of the '
             'unknown parameters is sought. This can be achieved using Markov '
             'chain Monte Carlo. On some occasions it is possible to deduce '
             'the form of the unknown system matrices in terms of a small '
             'number of scalar parameters, by considering the underlying '
             'physical processes involved. Here we study the case where this '
             'is not possible, and the entire matrices must be treated as '
             'unknowns. An efﬁcient Gibbs sampling algorithm exists for the '
             'basic formulation of linear model. We extend this to the mo

{'abstract': '',
 'keywords': ['functional magnetic resonance imaging data analysis; dynamic '
              'analysis'],
 'title': 'Dynamic discrimination analysis: A spatialâ\xa0\x8dtemporal SVM'}
--------------------------------------------------------------------------------
C:\Users\liste\OneDrive\Documents\schorlar\1-s2.0-S1053811911013012-main.pdf
{'/Author': '(Camilo Lamus)', '/CreationDate': "(D:20120908073116+08'00')", '/Creator': '(Elsevier)', '/CrossMarkDomains[1]': '(elsevier.com)', '/CrossMarkDomains[2]': '(sciencedirect.com)', '/CrossmarkDomainExclusive': '(true)', '/CrossmarkMajorVersionDate': '(2010-04-23)', '/ElsevierWebPDFSpecifications': '(6.2)', '/Keywords': '(MEG/EEG;  Source localization;  Inverse problem;  Dynamic spatiotemporal modeling)', '/ModDate': "(D:20140423184139+08'00')", '/Producer': '(Acrobat Distiller 10.0.0 \\(Windows\\))', '/Subject': '(NeuroImage, 63 \\(2012\\) 894-909. doi:10.1016/j.neuroimage.2011.11.020)', '/Title': '(A spatiotemporal dynamic d

{'abstract': 'MEG/EEG are non-invasive imaging techniques that record brain '
             'activity with high temporal resolution. However, estimation of '
             'brain source currents from surface recordings requires solving '
             'an ill-conditioned in- verse problem. Converging lines of '
             'evidence in neuroscience, from neuronal network models to '
             'resting-state imaging and neurophysiology, suggest that cortical '
             'activation is a distributed spatiotemporal dynamic process, '
             'supported by both local and long-distance neuroanatomic '
             'connections. Because spatiotemporal dy- namics of this kind are '
             'central to brain physiology, inverse solutions could be improved '
             'by incorporating models of these dynamics. In this article, we '
             'present a model for cortical activity based on nearest- neighbor '
             'autoregression that incorporates local spatiotempora

...

{'abstract': '', 'keywords': [], 'title': ''}
--------------------------------------------------------------------------------
C:\Users\liste\OneDrive\Documents\schorlar\302034.full.pdf
{'/Author': '(WIN 7)', '/CreationDate': "(D:20180416120158+04'30')", '/Creator': '(PScript5.dll Version 5.2.2)', '/ModDate': "(D:20190213221719-08'00')", '/Producer': '(Acrobat Distiller 11.0 \\(Windows\\))', '/Title': '(Beyond Core Object Recognition: Recurrent processes account for object recognition under occlusion)'}
[-- It is an ieee paper. --]
{'abstract': '', 'keywords': [], 'title': ''}
--------------------------------------------------------------------------------
C:\Users\liste\OneDrive\Documents\schorlar\358036.full.pdf
{'/Author': '()', '/CreationDate': '(D:20180628094725Z)', '/Creator': '(LaTeX with hyperref package)', '/Keywords': '()', '/ModDate': "(D:20190219223210-08'00')", '/PTEX.Fullbanner': '(This is pdfTeX, Version 3.14159265-2.6-1.40.18 \\(TeX Live 2017\\) kpathsea version 6.2.3)'

{'abstract': 'The application space for brain–computer interface (BCI) '
             'technologies is rapidly expanding with improvements in '
             'technology. However, most real-time BCIs require extensive '
             'individualized calibration prior to use, and systems often have '
             'to be recalibrated to account for changes in the neural signals '
             'due to a variety of factors including changes in human state, '
             'the surrounding environment, and task conditions. Novel '
             'approaches to reduce calibration time or effort will '
             'dramatically improve the usability of BCI systems. Active '
             'Learning (AL) is an iterative semi-supervised learning technique '
             'for learning in situations in which data may be abundant, but '
             'labels for the data are difﬁcult or expensive to obtain. In this '
             'paper, we apply AL to a simulated BCI system for target '
             'id

{'abstract': 'We describe BrainSync, an orthogonal transform that allows '
             'direct comparison of resting fMRI (rfMRI) time-series across '
             'subjects. For this purpose, we exploit the geometry of the rfMRI '
             'signal space to propose a novel orthogonal transformation that '
             'synchronizes rfMRI time-series across sessions and subjects. '
             'When synchronized, rfMRI signals become approximately equal at '
             'homologous locations across subjects. The method is based on the '
             'observation that rfMRI data exhibit similar con- nectivity '
             'patterns across subjects, as reﬂected in the pairwise '
             'correlations between different brain regions. We show that if '
             'the data for two subjects have similar correlation patterns then '
             'their time courses can be approximately synchronized by an '
             'orthogonal transformation. This transform is unique, inver

{'abstract': 'Neuroimaging studies have shown that the neural mechanisms of '
             'motor imagery (MI) overlap substantially with the mechanisms of '
             'motor execution (ME). Surprisingly, however, the role of several '
             'regions of the motor circuit- ry in MI remains controversial, a '
             'variability that may be due to differences in neuroimaging '
             'techniques, MI train- ing, instruction types, or tasks used to '
             'evoke MI. The objectives of this study were twofold: (i) to '
             'design a novel task that reliably invokes MI, provides a '
             'reliable behavioral measure of MI performance, and is '
             'transferable across imaging modalities; and (ii) to measure the '
             'common and differential activations for MI and ME with '
             'functional magnetic resonance imaging (fMRI) and '
             'magnetoencephalography (MEG). We present a task in which it is '
             

KeyboardInterrupt: 